In [ ]:
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
import math

# Curve fitting


This notebook illustrates linear basis function models. By experimenting with the code, you can explore the following aspects of these models:

* How different basis functions yield different predictions
* How complex models may explain noise instead of just the underlying pattern
* The effect of regularization
* Making predictions with uncertainty by defining prior distributions on the parameters.


### Synthetic Data

We generate synthetic data to which we fit the models. The data-generating function is $(f(x) = \sin(2 \pi x) + 1)$. Normally distributed noise $N(0, \beta)$ is added to the generated data.

In [ ]:
np.random.seed(42)
beta = 0.2

xs = np.array([-1.0, -0.9, -0.8, -0.75, -0.1, 0.1, 0.2, 0.3, 0.5, 0.8, 0.9, 1.0])
def true_fun(xs): 
    return np.sin(2 * math.pi * xs) + 1

ys = true_fun(xs) + stats.norm(0, beta).rvs(len(xs))

In [ ]:
plt.plot(xs, ys, 'o')
plt.ylim([-2,4])

## Linear Basis Function Models

The following function takes a list of basis functions and the data, 
and computes the parameters of the model. The parameters are calculated by simply solving the normal equations.

In [ ]:

def solve_regression(basis_functions, xs, ys):
    """ takes a list of basis_functions as well as the training data (xs, ys) and solves the 
        corresponding regression problem using the normal equation """
    m = len(basis_functions)
    
    # Matrix containing the basis functions evaluated at the input points
    Phi = np.zeros((len(xs), m))  
    for j in range(0, m):
        Phi[:,j] = basis_functions[j](xs)
        
    # small additional regularization term to improve numerical stability
    reg = 1e-10 * np.eye(len(basis_functions))
    w = np.linalg.inv(Phi.transpose() @ Phi + reg) @ Phi.transpose()

    return w @ ys

The following function is used to plot the solution.

In [ ]:
def plot_solution(w, basis_functions, xs, ys, show_basis = True, show_gt = True):
    """
    plots the basis function and the solutions for the (fitted) parameters w.
    xs and ys contain the data. 
    The parameter show_gt and show_basis determine whether the ground truth 
    and basis functions are plotted. 
    """
    # plot in the interval -1, 1
    x_values = np.linspace(-1, 1, 200)
    y_values = np.zeros(len(xs))
    
    # Compute the solution
    Phi = np.zeros((len(x_values), len(basis_functions)))
    for j in range(0, len(basis_functions)):
        Phi[:,j] = basis_functions[j](x_values)
    y_values = Phi @ w
    
    # Plot the best prediction
    plt.plot(xs, ys, 'o')    
    plt.plot(x_values, y_values, 'k', label="predicted")

    
    # Plot the basis fucntions
    if show_basis:
        for j in range(0, len(basis_functions)):
            plt.plot(x_values, basis_functions[j](x_values) * w[j], ':', color="grey")
  
    if show_gt:
        plt.plot(x_values,true_fun(x_values), '--r', label="ground truth")
    
    plt.xlabel("X")
    plt.ylabel("Y")
    plt.ylim([-2, 4])
    plt.legend()

Let's experiment with some polynomials:

In [ ]:
m = 15
basis_functions = [lambda x, i=i : np.power(x, i) for i in list(range(0,m+1))]
ws = solve_regression(basis_functions, xs, ys)
plot_solution(ws, basis_functions, xs, ys, show_gt = False)

Radial basis function are local basis functions and have better properties than 
polynomials. 

In [ ]:
s = 0.05
basis_functions = [lambda x, mu=mu : np.exp(-np.power(x-mu, 2)/s) for mu in xs ]
ws = solve_regression(basis_functions, xs, ys)
plot_solution(ws, basis_functions, xs, ys, show_gt = False)

### Prefering simple solutions: Ridge regression

To enforce simple solutions that do not learn the noise even with flexible models (many basis functions), we penalize solutions where the parameters are large. Thus, we solve $\min_w \sum_{i=0}^n (f(x_i,w)-y_i) + \alpha \| w \|^2$.

In [ ]:
  
def solve_regression_ridge(basis_functions, xs, ys, alpha):
    """ takes a list of basis_functions as well as the training data (xs, ys) and solves the 
        corresponding regression problem using least squares regression """
    m = len(basis_functions)
    Phi = np.zeros((len(xs), m))
    for j in range(0, m):
        Phi[:,j] = basis_functions[j](xs)
    w = np.linalg.inv(Phi.transpose() @ Phi + alpha * np.eye(len(basis_functions))) @ Phi.transpose()

    return w @ ys

Let's play with the regularization parameter $\alpha$ and observe what is happening to the 
basis functions.

In [ ]:
s = 0.05
alpha = 1e-1
basis_functions = [lambda x, mu=mu : np.exp(-np.power(x-mu, 2)/s) for mu in xs ]
ws = solve_regression_ridge(basis_functions, xs, ys, alpha)
plot_solution(ws, basis_functions, xs, ys)

### Regression using neural networks

Finally, we are using a neural network to fit the data. We are not implementing that ourselves, but using scikit learn. 

In [ ]:
from sklearn.neural_network import MLPRegressor

We define a Neural Network using a hidden layer of size 20 and and 'tanh' as an activation function. We specify the number of iterations for the optimizer. A tolerance of 0 tells the
optimizer not to stop before the maximum number of iterations. 

In [ ]:
regr = MLPRegressor(hidden_layer_sizes = (20,), activation='tanh', tol=0, max_iter = 30000)

The input to the `fit` function of the regression is a 2D array, with the individual training examples in the rows. We reshape our original data and call the `fit` function. 

In [ ]:
xs_array = np.array(xs).reshape((len(xs), 1))
regr.fit(xs_array, ys)

We plot the result to see how the neural network does.

In [ ]:
x_vals = np.linspace(-1, 1, 100).reshape((100, 1))
plt.plot(xs, ys, 'o')
plt.plot(x_vals, regr.predict(x_vals))